# Task 1: Setup your development environment and store the test video locally (10 points)

## Evironment

I'm using a Docker-based environment that takes advantage of my NVIDIA GPU. To learn more about the configuration for this container, please see docker-compose.yaml, Dockerfile, and requirements.txt files. In short, I'm using a tensorflow base image that supports Jupyter notebooks and GPU usage, installing dependencies, and exposing a port so I can access the notebook within the container through my host machine's browser. I'm also mounting a volume to persist any changes made in the container. To set this up, I also had to install the appropriate GPU drivers, WSL2, and Docker Desktop and did so by following [this tutorial](https://www.youtube.com/watch?v=YozfiLI1ogY&t=717s&ab_channel=KNuggies).

In [8]:
import torch

In [9]:
print(torch.cuda.is_available())

True


## Store Test Video Locally

In [10]:
from pytube import YouTube
import re
import os

In [11]:
def download(url):
    if not os.path.exists("videos"):
        os.makedirs("videos")
    pattern = r"v=([^&]+)"
    videoId = re.search(pattern, url).group(1)
    yt = YouTube(url)
    stream = yt.streams.filter(progressive=True).get_by_itag(22)
    stream.download(filename="videos/" + videoId+".mp4")

In [12]:
urls = ["https://www.youtube.com/watch?v=WeF4wpw7w9k&t=44s&ab_channel=PantelisMonogioudis",
        "https://www.youtube.com/watch?v=2NFwY15tRtA&ab_channel=PantelisMonogioudis",
        "https://www.youtube.com/watch?v=5dRramZVu2Q&ab_channel=R2bEEaton"]

In [13]:
for url in urls:
    download(url)

# Task 2: Object Detection (40 points)

In [14]:
import ultralytics
import ultralyticsplus

In [15]:
from ultralyticsplus import YOLO, render_result
model = YOLO('mshamrai/yolov8s-visdrone')

100%|██████████| 6.23M/6.23M [00:00<00:00, 71.0MB/s]


config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

best.pt:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

In [16]:
import cv2
import numpy as np
from PIL import Image

In [17]:
output_directory = "object_detected_videos"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [18]:
def parse(inFile, outFile):
    inVideo = cv2.VideoCapture(inFile)
    fps = inVideo.get(cv2.CAP_PROP_FPS)
    frame_size = (int(inVideo.get(cv2.CAP_PROP_FRAME_WIDTH)), int(inVideo.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    outVideo = cv2.VideoWriter(outFile, cv2.VideoWriter_fourcc(*'mp4v'), fps, frame_size)
    frameCount = 0
    while True:
        success, frame = inVideo.read()
        frameCount += 1
        if not success: break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model.predict(frame)
        render = render_result(model=model, image=rgb_frame, result=results[0])
        outFrame = cv2.cvtColor(np.array(render), cv2.COLOR_RGB2BGR)
        outVideo.write(np.array(outFrame))
    inVideo.release()
    outVideo.release()

In [19]:
video_paths = os.listdir("videos")

In [ ]:
for path in video_paths:
    videoId = path.split('.')[0]
    parse("videos/" + path, output_directory + "/" + videoId + ".mp4")


0: 352x640 (no detections), 97.1ms
Speed: 5.7ms preprocess, 97.1ms inference, 30.2ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 6.5ms
Speed: 1.9ms preprocess, 6.5ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 7.2ms
Speed: 1.6ms preprocess, 7.2ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 7.5ms
Speed: 3.2ms preprocess, 7.5ms inference, 0.9ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 6.9ms
Speed: 1.5ms preprocess, 6.9ms inference, 0.8ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 7.0ms
Speed: 2.8ms preprocess, 7.0ms inference, 0.8ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 8.5ms
Speed: 3.7ms preprocess, 8.5ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 motor, 8.5ms
Speed: 2.0ms preprocess, 8.5ms inference, 69.0ms post

This code used a model pretrained on a drone dataset as an object detector. I then parsed through each video's frames, using the object detector on each one. Using the output of the object detector, I created new videos with the bounding box and label to show that object detection was successful. Given that GitHub doesn't store large files, I chose to host these videos on YouTube.
- https://youtu.be/KOMcCO_nfD0
- https://youtu.be/Mt_TAALjJQg
- https://youtu.be/YJ3OSMPVL88

## Task 3: Kalman Filter (50 points)

I will be using [this video](https://www.youtube.com/watch?v=KOMcCO_nfD0&ab_channel=SaumyaDwivedi) as my object detector performs the best on it.

In [79]:
from filterpy.kalman import KalmanFilter
import matplotlib.pyplot as plt

In [107]:
def kalman(inFile):
    f = KalmanFilter(dim_x=4, dim_z=2)
    initialized = False
    inVideo = cv2.VideoCapture(inFile)
    fps = inVideo.get(cv2.CAP_PROP_FPS)
    frame_size = (int(inVideo.get(cv2.CAP_PROP_FRAME_WIDTH)), int(inVideo.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    outVideo = cv2.VideoWriter("kalman.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, frame_size)
    frameCount = 0
    while True:
        success, frame = inVideo.read()
        frameCount += 1
        if not success: break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model.predict(frame)
        boxes = results[0].boxes
        if (3 in list(boxes.cls)):
            xyxy = boxes.xyxy[0]
            position = [int((xyxy[0] + xyxy[2]) / 2), int((xyxy[1] + xyxy[3]) / 2)]
            if initialized:
                f.update(position)
                f.predict()
                estimated_state = f.x[:2]
                slope = (estimated_state[1] - position[1]) / (estimated_state[0] - position[0])
                extension_distance = 100
                new_x = estimated_state[0] - np.sign(slope) * extension_distance / np.sqrt(1 + slope**2)
                new_y = estimated_state[1] - slope * (new_x - estimated_state[0])
                lineFrame = cv2.line(rgb_frame, position, (int(new_x), int(new_y)), (255, 0, 0), thickness=3)
                outFrame = cv2.rectangle(lineFrame, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (255, 255, 255), thickness=5)
                outVideo.write(cv2.cvtColor(outFrame, cv2.COLOR_RGB2BGR))
            else:
                xyxy = boxes.xyxy
                position.extend([0, 0])
                f.x = np.array(position)
                f.P = np.eye(4) * 1000
                dt = 1.0
                f.F = np.array([[1, 0, dt, 0],
                                 [0, 1, 0, dt],
                                 [0, 0, 1, 0],
                                 [0, 0, 0, 1]])
                f.H = np.array([[1, 0, 0, 0],
                                [0, 1, 0, 0]])
                f.Q = np.eye(4) * 0.01
                f.R = np.eye(2) * 10
                f.predict()
                initialized = True
                outVideo.write(cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR))
        else:
            outVideo.write(cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR))
    inVideo.release()
    outVideo.release()
kalman('videos/WeF4wpw7w9k.mp4')


0: 352x640 (no detections), 47.3ms
Speed: 3.6ms preprocess, 47.3ms inference, 2.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 47.5ms
Speed: 3.1ms preprocess, 47.5ms inference, 1.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 43.9ms
Speed: 3.1ms preprocess, 43.9ms inference, 1.4ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 42.1ms
Speed: 2.9ms preprocess, 42.1ms inference, 1.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 42.4ms
Speed: 2.9ms preprocess, 42.4ms inference, 1.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 42.3ms
Speed: 2.8ms preprocess, 42.3ms inference, 2.1ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 9.5ms
Speed: 1.7ms preprocess, 9.5ms inference, 1.1ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 motor, 7.7ms
Speed: 1.5ms preprocess, 7.7ms inference, 1.

For Task 3, I reused a lot of the logic from Task 2. I parsed the video's frames and used the object detector on each one to draw the bounding box for the car if it was detected. If the Kalman filter was not initialized, I also used the output from the object detector to to perform this initialization and then make a prediction, drawing it in the frame. If the Kalman filter was initialized, I used the output from the object detector to update the Kalman filter and then make a prediction, drawing it in the frame. With all of these frames, I created a new video. Once again, I chose to host this video on YouTube.
- https://youtu.be/l8anYbV6NuU

A potential way to address false positives is deciding that if the position detected deviates significantly from the estimate by the Kalman filter, we can flag it as false positive. This makes sense as objects don't teleport, instead traveling through space through non-zero velocity.